In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!cp -r "/content/gdrive/MyDrive/final_antispoofing.zip" "/content"

In [3]:
import zipfile
archive = zipfile.ZipFile('/content/final_antispoofing.zip')
archive.extractall('/content')

In [4]:
dataset_dir = '/content/final_antispoofing'
train_dataset_dir = '/content/final_antispoofing/train'
test_dataset_dir = '/content/final_antispoofing/test'

In [5]:
import os
os.mkdir('/content/antispoofing_dataset')
os.mkdir('/content/antispoofing_dataset/train')
os.mkdir('/content/antispoofing_dataset/test')
os.mkdir('/content/antispoofing_dataset/train/real')
os.mkdir('/content/antispoofing_dataset/train/spoof')
os.mkdir('/content/antispoofing_dataset/test/real')
os.mkdir('/content/antispoofing_dataset/test/spoof')

In [6]:
train_dir='/content/antispoofing_dataset/train'
test_dir = '/content/antispoofing_dataset/test'

In [7]:
import shutil
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [8]:
def train_test_splits(data_directory):
  for split_type in os.listdir(data_directory):
    path_to_split_type = os.path.join(data_directory,split_type)
    for category in os.listdir(path_to_split_type):
      path_to_category = os.path.join(path_to_split_type,category)
      for subject in os.listdir(path_to_category):
        path_to_subject = os.path.join(path_to_category,subject)
        for img in os.listdir(path_to_subject):
          if split_type == 'train':
            shutil.copy(os.path.join(path_to_subject,img),os.path.join(train_dir,category,img))
          else:
            shutil.copy(os.path.join(path_to_subject,img),os.path.join(test_dir,category,img))

In [9]:
train_test_splits(data_directory=dataset_dir)

In [10]:
categories = ['real','spoof']

In [11]:
print("---------------------Exploring Training Datasets--------------------")
for category in categories:
  path = os.path.join(train_dir,category)
  if category == 'real':
    r1 = len(os.listdir(path))
  else:
    s1 = len(os.listdir(path))
  print("There are {} images in {} directory".format(len(os.listdir(path)),category))
print("There are {} total images in training directory".format(r1+s1))

print("-----------------------Exploring Testing Datasets-------------------------")
for category in categories:
  path = os.path.join(test_dir,category)
  if category == 'real':
    r2 = len(os.listdir(path))
  else:
    s2 = len(os.listdir(path))
  print("There are {} images in {} directory".format(len(os.listdir(path)),category))
print("There are {} total images in testing directory".format(r2+s2))

---------------------Exploring Training Datasets--------------------
There are 2102 images in real directory
There are 2118 images in spoof directory
There are 4220 total images in training directory
-----------------------Exploring Testing Datasets-------------------------
There are 477 images in real directory
There are 474 images in spoof directory
There are 951 total images in testing directory


**MODEL**

In [12]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [13]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json

In [14]:
train_datagen = ImageDataGenerator(brightness_range=(0.8,1.2),rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,fill_mode='nearest',shear_range=0.2,zoom_range=0.3,rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(160,160),color_mode='rgb',
                                                    class_mode='binary',batch_size=30,shuffle=True)

Found 4220 images belonging to 2 classes.


In [16]:
valid_generator = valid_datagen.flow_from_directory(test_dir,target_size=(160,160),color_mode='rgb',
                                                    class_mode='binary',batch_size=30)

Found 951 images belonging to 2 classes.


In [17]:
cnn = models.Sequential([
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(160, 160, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [18]:
cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
cnn.fit(train_generator, epochs=5)

Epoch 1/5
141/141 [==============================] - 221s 2s/step - loss: 0.6617 - accuracy: 0.7889
Epoch 2/5
141/141 [==============================] - 231s 2s/step - loss: 0.3332 - accuracy: 0.8552
Epoch 3/5
141/141 [==============================] - 224s 2s/step - loss: 0.2989 - accuracy: 0.8739
Epoch 4/5
141/141 [==============================] - 217s 2s/step - loss: 0.2994 - accuracy: 0.8742
Epoch 5/5
141/141 [==============================] - 231s 2s/step - loss: 0.2875 - accuracy: 0.8775


In [20]:
cnn.evaluate(valid_generator)

32/32 [==============================] - 13s 405ms/step - loss: 0.1657 - accuracy: 0.9201


[0.1656750589609146, 0.9200841188430786]

In [21]:
model_json = cnn.to_json()
with open("cnn-model.json", "w") as json_file:
    json_file.write(model_json)

In [22]:
cnn.save("model.h5")